## PATIENT_FOLLOW_UP to OMOP 

In [21]:
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

def get_random_value(val):
    if isinstance(val, tuple):
        return np.random.choice(val)
    return val

#### DUDAS DE CIERTOS MAPEOS

Hay tablas que mapean a valores ya metidos antes. Ver posibles problemas de esto, tenerlo en cuenta.

#### LINKS RECOMENDADOS

El siguiente link nos habla sobre la falta de un valor en la columna visit_start_date de OMOP. No se puede poner cualquier valor como "default". Hay que especificar el valor correcto. Imagino que esto aplica no solo a esta columna, sino al resto de columnas que contengan una fecha en específico, como la columna "episode_end_date" de la tabla EPISODE de OMOP que también tenemos que mapear aquí: 

https://forums.ohdsi.org/t/missing-visit-start-date/16470 

### Extracción de datos PatientFollowUp

In [22]:
# Leer datos del archivo CSV
df_patients_FollowUp_IDEA4RC = pd.read_csv("./IDEA4RC-data/patientsFollowUpIDEA4RC.csv")

In [23]:
df_patients_FollowUp_IDEA4RC.head(3)

,Patient,Status at last follow-up,Patient Follow Up date,New cancer diagnosis,Date of new cancer diagnosis,New cancer topography,Last Contact
0,45,0,0,4188540,0,44498973,0
1,20,0,0,4188539,0,36534215,0
2,35,0,0,4188539,0,44498973,0


- "Patient": Supongo que son los mismos "id" de la tabla "Patient". Es decir, en OMOP serán los mismos valores que "person_id".

- "Status at last follow-up": pongo 0 como "default" ya que no hay valores especificados para los vocabularios del EXCEL

- "Patient Follow Up date": pongo 0 como "default" ya que en IDEA4RC no se especifíca nada. Ver link de arriba donde explica que valor debe de ir en la columna visit_start_date de la tabla VISIT de OMOP

- "New cancer diagnosis": está bien. Ver si estos vocabularios figuran en la tabla Patient. Sino, añadir a OMOP de forma igual que en la de PATIENT, con su person_id y observation_concept_id

- "Date of new cancer diagnosis": 0 por defecto ya que no hay valor en IDEA4RC. En OMOP se especifíca que ese valor sale de la tabla Condition_Occurence. Ver de donde sale.

- "New cancer topography": Tiene referencia numérica de vocabulario, pero no se especifica en que tabla de OMOP añadir esta columna.

- "Last Contact": 0 por defecto, sin valor en IDEA4RC. Añadir como date a la tabla EPISODE. Imagino que no puede ser un valor arbitrario, como lo que se explica en "Patient Follow Up date".

In [24]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
cur = conn.cursor()

df_patients_follow_up.head(5)

vocab_uploader-1  | CREATE TABLE IF NOT EXISTS omopcdm.VISIT_OCCURRENCE
vocab_uploader-1  | (
vocab_uploader-1  |     visit_occurrence_id           integer     NOT NULL,
vocab_uploader-1  |     person_id                     integer     NOT NULL,
vocab_uploader-1  |     visit_concept_id              integer     NOT NULL,
vocab_uploader-1  |     visit_start_date              date        NOT NULL,
vocab_uploader-1  |     visit_end_date                date        NOT NULL,
vocab_uploader-1  |     visit_type_concept_id         Integer     NOT NULL,

### PatientFollowUp to VISIT_OCCURRENCE table

In [28]:
patients_Foll_Up_column_names = []


# Convertir todas las columnas a tipo 'object'
df_patients_FollowUp_IDEA4RC = df_patients_FollowUp_IDEA4RC.astype(object)

# Dataframe para columnas y datos necesarios no disponibles o no definidos en Idea4RC
df_tables = pd.DataFrame()
df_tables.index = range(len(df_patient_IDEA4RC))

df_tables["Date"] = [datetime.now().date()] * len(df_patients_FollowUp_IDEA4RC)
    
df_tables['condition_type'] = [42894222 if random.random() > 0.5 else 0 for _ in range(len(df_tables))]
df_tables["condition_type"] = df_tables["condition_type"].astype(object)

# Crear una lista para almacenar los valores que se van a insertar
values_to_insert = []

for _, row in df_patients_FollowUp_IDEA4RC.iterrows():
    person_id = row['Patient']
    date_value = df_tables.loc[_]['Date'].strftime('%Y-%m-%d')  # Obtener el valor de 'Date' del DataFrame df_tables
    condition_type_value = df_tables.loc[_]['condition_type'] 
    for column in patients_Foll_Up_column_names:
        condition_value = row[column]
        values_to_insert.append((person_id, condition_value, date_value, condition_type_value))
        

condition_ids = list(range(len(values_to_insert) + 1))  # Crear una lista desde 0 hasta el máximo (inclusive)

# Crear la consulta SQL para la inserción
sql = """
    INSERT INTO omopcdm.condition_occurrence (visit_occurrence_id,  visit_concept_id, visit_start_date, visit_end_Date, visit_type_concept_id)
    VALUES (%s, %s, %s, %s, %s)
"""

# Combinar condition_ids con values_to_insert
data_to_insert = [(condition_id, *values) for condition_id, values in zip(condition_ids, values_to_insert)]

# Ejecutar la consulta SQL con los valores no nulos
with conn.cursor() as cur:
    cur.executemany(sql, data_to_insert)

# Confirmar la transacción
conn.commit()

# # Cerrar la conexión
# conn.close()

In [ ]:
vocab_uploader-1  | CREATE TABLE IF NOT EXISTS omopcdm.EPISODE
vocab_uploader-1  | (
vocab_uploader-1  |     episode_id                bigint      NOT NULL,
vocab_uploader-1  |     person_id                 bigint      NOT NULL,
vocab_uploader-1  |     episode_concept_id        integer     NOT NULL,
vocab_uploader-1  |     episode_start_date        date        NOT NULL,
vocab_uploader-1  |     episode_object_concept_id integer     NOT NULL,
vocab_uploader-1  |     episode_type_concept_id   integer     NOT NULL,
vocab_uploader-1  | );

### PatientFollowUp to EPISODE table

episode_object_concept_id???

In [ ]:
df_tables['condition_type'] = [42894222 if random.random() > 0.5 else 0 for _ in range(len(df_tables))]
df_tables["condition_type"] = df_tables["condition_type"].astype(object)

# Crear una lista para almacenar los valores que se van a insertar
values_to_insert = []

for _, row in df_patients_FollowUp_IDEA4RC.iterrows():
    person_id = row['Patient']
    date_value = df_tables.loc[_]['Date'].strftime('%Y-%m-%d')  # Obtener el valor de 'Date' del DataFrame df_tables
    condition_type_value = df_tables.loc[_]['condition_type'] 
    for column in patient_column_condition_names:
        condition_value = row[column]
        values_to_insert.append((person_id, condition_value, date_value, condition_type_value))
        

condition_ids = list(range(len(values_to_insert) + 1))  # Crear una lista desde 0 hasta el máximo (inclusive)

# Crear la consulta SQL para la inserción
sql = """
    INSERT INTO omopcdm.condition_occurrence (episode_id, person_id, episode_concept_id, episode_start_date, episode_object_concept_id, episode_type_concept_id)
    VALUES (%s, %s, %s, %s, %s)
"""

# Combinar condition_ids con values_to_insert
data_to_insert = [(condition_id, *values) for condition_id, values in zip(condition_ids, values_to_insert)]

# Ejecutar la consulta SQL con los valores no nulos
with conn.cursor() as cur:
    cur.executemany(sql, data_to_insert)

# Confirmar la transacción
conn.commit()

# # Cerrar la conexión
# conn.close()

### PatientFollowUp to OBSERVATION table

### PatientFollowUp to CONDITION_OCCURRENCE table?????